In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# 🔹 처리할 연도 및 월 목록 정의 (2023년 6월~12월)
year = "2023"
months = [f"{i:02d}" for i in range(1, 13)]  # 01 ~ 12 형식으로 월 생성

In [ ]:
# 🔹 반복문을 사용하여 여러 개의 파일을 처리
for month in months:
    csv_file_path = f"/content/drive/MyDrive/원본/TN_MNBY_TASTE_RSTRNT_FD_GRW_ADVRTS_EXCUT_DATABASE_INFO_{year}{month}.csv"

    try:
        # 데이터 불러오기
        data = pd.read_csv(csv_file_path, encoding="utf-8-sig")

        # 🔹 Step 1: 불필요한 열 삭제
        columns_to_drop = [
            'ADVRTS_CD', 'ADVRTS_DE', 'ADVRTS_CL_NM', 'ADVRTS_BRDCST_AREA_NM', 'ADVRTSR_NM', 'ADVRTSR_CD',
            'BRAND_NM', 'BRAND_CD', 'INDUTY_LCLAS_NM', 'INDUTY_LCLAS_CD', 'INDUTY_MLSFC_NM', 'INDUTY_MLSFC_CD',
            'INDUTY_SCLAS_CD', 'PROGRM_BRDCST_AREA_NM', 'ADVRTS_PROGRM_NM', 'PROGRM_BEGIN_TIME', 'PROGRM_END_TIME'
        ]
        data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors="ignore")

        # 🔹 Step 2: 모든 노출률(`NT01_EXPSR_REACH_RT` ~ `NT05_EXPSR_REACH_RT`)이 NaN 또는 0인 행 제거
        exposure_cols = ['NT01_EXPSR_REACH_RT', 'NT02_EXPSR_REACH_RT',
                         'NT03_EXPSR_REACH_RT', 'NT04_EXPSR_REACH_RT', 'NT05_EXPSR_REACH_RT']
        data = data[~(data[exposure_cols].isna().all(axis=1) | (data[exposure_cols] == 0).all(axis=1))]

        # 🔹 Step 3: `AREA_NM`이 `"전국"`인 행 제거
        data = data[data['AREA_NM'] != "전국"]

        # 🔹 Step 4: `CHNNEL_NM`이 `"MBC"`, `"SBS"`, `"KBS"` 중 하나인 경우만 유지
        data = data[data['CHNNEL_NM'].isin(["MBC", "SBS", "KBS2"])]

        # 🔹 Step 5: `frequency` 컬럼 생성 (노출 빈도 계산)
        data['exposure_count'] = data[exposure_cols].gt(0).sum(axis=1)  # 0이 아닌 노출률 개수 계산
        data['frequency'] = data['exposure_count'].apply(lambda x: 0 if x < 3 else 1)  # 3회 미만이면 0, 아니면 1

        # 임시로 사용한 `exposure_count` 컬럼 삭제
        data.drop(columns=['exposure_count'], inplace=True)

        # 🔹 Step 6: `NT01` ~ `NT05` 컬럼 삭제
        data.drop(columns=exposure_cols, inplace=True)

        # 🔹 Step 7: 처리된 데이터 저장
        processed_file_path = f"/content/drive/MyDrive/월별/{year}_{month}.csv"
        data.to_csv(processed_file_path, index=False, encoding="utf-8-sig")

        print(f"✔ {year}_{month}.csv 파일 저장 완료!")

    except FileNotFoundError:
        print(f"⚠ {csv_file_path} 파일이 존재하지 않습니다. 확인 후 다시 실행하세요.")


✔ 2023_01.csv 파일 저장 완료!
✔ 2023_02.csv 파일 저장 완료!
✔ 2023_03.csv 파일 저장 완료!
✔ 2023_04.csv 파일 저장 완료!
✔ 2023_05.csv 파일 저장 완료!


<ipython-input-9-502444709fae>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exposure_count'] = data[exposure_cols].gt(0).sum(axis=1)  # 0이 아닌 노출률 개수 계산
<ipython-input-9-502444709fae>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['frequency'] = data['exposure_count'].apply(lambda x: 0 if x < 3 else 1)  # 3회 미만이면 0, 아니면 1
<ipython-input-9-502444709fae>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

✔ 2023_06.csv 파일 저장 완료!
✔ 2023_07.csv 파일 저장 완료!
✔ 2023_08.csv 파일 저장 완료!
✔ 2023_09.csv 파일 저장 완료!
✔ 2023_10.csv 파일 저장 완료!
✔ 2023_11.csv 파일 저장 완료!
✔ 2023_12.csv 파일 저장 완료!


In [ ]:
import os

# 빈 리스트 생성 (각 데이터프레임 저장용)
dataframes = []

# 🔹 각 파일의 크기(shape) 출력 및 데이터 읽기
for month in months:
    csv_file_path = f"/content/drive/MyDrive/월별/{year}_{month}.csv"  # 파일 경로 설정

    if os.path.exists(csv_file_path):  # 파일 존재 여부 확인
        # 데이터 불러오기
        data = pd.read_csv(csv_file_path, encoding="utf-8-sig")

        # 데이터 크기 출력
        print(f"📊 {year}_{month}.csv 파일 크기: {data.shape}")

        # 데이터프레임 리스트에 추가
        dataframes.append(data)
    else:
        print(f"⚠ {csv_file_path} 파일이 존재하지 않습니다.")

# 🔹 모든 데이터를 하나의 데이터프레임으로 병합
if dataframes:
    merged_data = pd.concat(dataframes, ignore_index=True)

    # 병합된 데이터 저장
    merged_file_path = "/content/drive/MyDrive/merged2023.csv"
    merged_data.to_csv(merged_file_path, index=False, encoding="utf-8-sig")

    print(f"✔ 병합된 데이터 저장 완료: {merged_file_path}")

else:
    print("❌ 병합할 데이터가 없습니다. 파일을 확인해주세요.")

📊 2023_01.csv 파일 크기: (80166, 14)
📊 2023_02.csv 파일 크기: (72385, 14)
📊 2023_03.csv 파일 크기: (80155, 14)
📊 2023_04.csv 파일 크기: (47324, 14)
📊 2023_05.csv 파일 크기: (90075, 14)
📊 2023_06.csv 파일 크기: (123997, 14)
📊 2023_07.csv 파일 크기: (58614, 14)
📊 2023_08.csv 파일 크기: (109292, 14)
📊 2023_09.csv 파일 크기: (128252, 14)
📊 2023_10.csv 파일 크기: (107156, 14)
📊 2023_11.csv 파일 크기: (89262, 14)
📊 2023_12.csv 파일 크기: (146934, 14)
✔ 병합된 데이터 저장 완료: /content/drive/MyDrive/merged2023.csv


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/merged2023.csv")
df.shape

(1133612, 14)